In [12]:
import warnings
warnings.filterwarnings("ignore")

In [13]:
import pandas as pd

In [14]:
customer = pd.read_csv("../input/customer_join.csv")
uselog_months = pd.read_csv("../input/use_log_months.csv")

In [15]:
year_months = list(uselog_months["年月"].unique())
uselog = pd.DataFrame()

for i in range(1, len(year_months)):
    tmp = uselog_months.loc[uselog_months["年月"]==year_months[i]]
    tmp.rename(columns={"count":"count_0"}, inplace=True)
    
    tmp_before = uselog_months.loc[uselog_months["年月"]==year_months[i-1]]
    del tmp_before["年月"]
    tmp_before.rename(columns={"count":"count_1"}, inplace=True)
    
    tmp = pd.merge(tmp, tmp_before, on="customer_id", how="left")
    uselog = pd.concat([uselog, tmp], ignore_index=True)

uselog.head()    

,年月,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0


In [17]:
from dateutil.relativedelta import relativedelta

exit_customer = customer.loc[customer["is_deleted"]==1]
exit_customer["exit_date"] = None
exit_customer["end_date"] = pd.to_datetime(exit_customer["end_date"])

for i in range(len(exit_customer)):
    exit_customer["exit_date"].iloc[i] = exit_customer["end_date"].iloc[i] - relativedelta(months=1)

exit_customer["年月"] = exit_customer["exit_date"].dt.strftime("%Y%m")
uselog["年月"] = uselog["年月"].astype(str)
exit_uselog = pd.merge(uselog, exit_customer, on=["customer_id", "年月"], how="left")

print(len(uselog))
exit_uselog.head()

33851


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201805,AS002855,5,4.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201805,AS009373,4,3.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201805,AS015233,7,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201805,AS015315,3,6.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201805,AS015739,5,7.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
exit_uselog = exit_uselog.dropna(subset=["name"])
print(len(exit_uselog))
print(len(exit_uselog["customer_id"].unique()))
exit_uselog.head()

1104
1104


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
19,201805,AS055680,3,3.0,XXXXX,C01,M,2018-03-01,2018-06-30,CA1,...,10500.0,通常,3.000000,3.0,3.0,3.0,0.0,2018-06-30,3.0,2018-05-30 00:00:00
57,201805,AS169823,2,3.0,XX,C01,M,2017-11-01,2018-06-30,CA1,...,10500.0,通常,3.000000,3.0,4.0,2.0,1.0,2018-06-30,7.0,2018-05-30 00:00:00
110,201805,AS305860,5,3.0,XXXX,C01,M,2017-06-01,2018-06-30,CA1,...,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,12.0,2018-05-30 00:00:00
128,201805,AS363699,5,3.0,XXXXX,C01,M,2018-02-01,2018-06-30,CA1,...,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,4.0,2018-05-30 00:00:00
147,201805,AS417696,1,4.0,XX,C03,F,2017-09-01,2018-06-30,CA1,...,6000.0,通常,2.000000,1.0,4.0,1.0,0.0,2018-06-30,9.0,2018-05-30 00:00:00


In [19]:
conti_customer = customer.loc[customer["is_deleted"]==0]
conti_uselog = pd.merge(uselog, conti_customer, on=["customer_id"], how="left")
print(len(conti_uselog))
conti_uselog = conti_uselog.dropna(subset=["name"])
print(len(conti_uselog))

33851
27422


In [21]:
conti_uselog = conti_uselog.sample(frac=1).reset_index(drop=True)
conti_uselog = conti_uselog.drop_duplicates(subset=["customer_id"])
print(len(conti_uselog))
conti_uselog.head()

2842


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201902,HD165212,4,5.0,XXXX,C03,M,2017-01-01,NaN,CA1,...,ナイト,6000.0,通常,5.250000,5.5,9.0,2.0,1.0,2019-04-30,27.0
1,201812,HD559223,8,7.0,XXXXXX,C01,F,2017-03-01,NaN,CA1,...,オールタイム,10500.0,通常,5.333333,5.0,8.0,3.0,1.0,2019-04-30,25.0
2,201808,IK913187,6,5.0,XXXX,C01,F,2017-02-01,NaN,CA1,...,オールタイム,10500.0,通常,5.000000,5.5,7.0,1.0,1.0,2019-04-30,26.0
3,201811,HI093998,11,NaN,XXXX,C03,M,2018-11-04,NaN,CA1,...,ナイト,6000.0,通常,9.000000,9.0,11.0,6.0,1.0,2019-04-30,5.0
4,201903,GD289362,7,6.0,XX,C02,F,2018-05-04,NaN,CA2,...,デイタイム,7500.0,入会費半額,8.090909,8.0,10.0,6.0,1.0,2019-04-30,11.0


In [22]:
predict_data = pd.concat([conti_uselog, exit_uselog], ignore_index=True)
print(len(predict_data))
predict_data.head()

3946


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201902,HD165212,4,5.0,XXXX,C03,M,2017-01-01,NaN,CA1,...,6000.0,通常,5.250000,5.5,9.0,2.0,1.0,2019-04-30,27.0,NaN
1,201812,HD559223,8,7.0,XXXXXX,C01,F,2017-03-01,NaN,CA1,...,10500.0,通常,5.333333,5.0,8.0,3.0,1.0,2019-04-30,25.0,NaN
2,201808,IK913187,6,5.0,XXXX,C01,F,2017-02-01,NaN,CA1,...,10500.0,通常,5.000000,5.5,7.0,1.0,1.0,2019-04-30,26.0,NaN
3,201811,HI093998,11,NaN,XXXX,C03,M,2018-11-04,NaN,CA1,...,6000.0,通常,9.000000,9.0,11.0,6.0,1.0,2019-04-30,5.0,NaN
4,201903,GD289362,7,6.0,XX,C02,F,2018-05-04,NaN,CA2,...,7500.0,入会費半額,8.090909,8.0,10.0,6.0,1.0,2019-04-30,11.0,NaN


In [ ]:
predict_data["period"] = 0
preditc_data["now_data"] = pd.to_datatime(predict_data["年月"])